# Clinical Profile Calculations on JHU Diabetes Sample
### Steph Howson, JHU/APL, Data Scientist

This notebook calculates fields to be generated for the Clinical Profiles model. Once the values are calculated, the results will be dynamically put into the model with the fhir.resources implementation. The Clinical Profiles Python specification was built using fhir-parser. These forked Github repositories can be found (currently not much was done to add desired features for Clinical Profiles in particular, but the templating captures much of the functionality needed):

https://github.com/stephanie-howson/fhir-parser

https://github.com/stephanie-howson/fhir.resources

The Clinical Profile Python FHIR Class definition can be found at:

https://github.com/stephanie-howson/fhir.resources/blob/master/fhir/resources/clinicalprofile.py

### Imports

In [1]:
import pandas as pd
import numpy as np
import scipy.stats as ss
import math
import dask.dataframe as dd
import sys

### Reading in data from SAFE

In [2]:
# Want to specify dtypes for performance
demographics_DOB = ['DOB']
demographics_dtypes = {'PatientID':np.int64, 'Gender':'category','Race':'category','Ethnicity':'category'}

labs_dates = ['Ordering_datetime','Result_datetime']
labs_dtypes = {'PatientID':np.int64, 'EncounterID':np.int64, 'Result_numeric':np.float64,'Lab_Name':'category',
               'Base_Name':'category','Loinc_Code':'category','LONG_COMMON_NAME':'category',
                'status':'category','Category':'category','GroupId':'category','unit':'category',
               'range':'category'}

diagnoses_hpo_dates = ['Entry_Date']
diagnoses_hpo_dtypes = {'PatientID':np.int64, 'icd_10':'category','icd_name':'category',
                       'hpo':'category','hpo_term':'category'}

encounter_dates = ['Encounter_date']
encounter_dtypes = {'PatientID': np.int64,'EncounterID': np.int64, 'Encounter_type':'category'}

meds_dates = ['Order_datetime','Start_date','End_date']
meds_dtypes = {'PatientID': np.int64,'EncounterID': np.int64, 'Medication_Name':'category','Dose':'category',
              'Route':'category', 'Frequency':'category', 'Quantity':'category', 'RXNorm':np.float64,'Therapeutic_Class':'category',
                   'Pharmaceutical_Class':'category', 'Pharmaceutical_Subclass':'category'}

procedure_dtypes = {'PatidentID':np.int64,'EncounterID':np.int64, 'Procedure_ID':np.int64,'Procedure_Code':'category',
                   'Procedure_Name':'category'}

In [3]:
df_demographics = pd.read_csv(r'S:\NCATS\Clinical_Profiles\clean_data\Diabetes\jh_diabetes_demographics.txt',sep='|',
                             dtype=demographics_dtypes, parse_dates=demographics_DOB)

In [5]:
print(sys.getsizeof(df_demographics)*10**(-9))

0.001366428


In [4]:
df_labs = pd.read_csv(r'S:\NCATS\Clinical_Profiles\clean_data\Diabetes\jh_diabetes_labs.txt',sep='|',
                     dtype=labs_dtypes, parse_dates=labs_dates)

In [7]:
print(sys.getsizeof(df_labs)*10**(-9))

0.473893587


In [8]:
df_diagnoses_hpo = pd.read_csv(r'S:\NCATS\Clinical_Profiles\clean_data\Diabetes\jh_diabetes_diagnoses_hpo.txt',sep='|',
                              dtype=diagnoses_hpo_dtypes, parse_dates=diagnoses_hpo_dates)

In [9]:
print(sys.getsizeof(df_diagnoses_hpo)*10**(-9))

0.005669864


In [10]:
df_encounter = pd.read_csv(r'S:\NCATS\Clinical_Profiles\clean_data\Diabetes\jh_diabetes_encounter.txt',sep='|',
                          dtype=encounter_dtypes, parse_dates=encounter_dates)

In [11]:
print(sys.getsizeof(df_encounter)*10**(-9))

0.03001293


In [12]:
df_meds = pd.read_csv(r'S:\NCATS\Clinical_Profiles\clean_data\Diabetes\jh_diabetes_meds.txt',sep='|',
                     dtype=meds_dtypes, parse_dates=meds_dates)

In [13]:
print(sys.getsizeof(df_meds)*10**(-9))

0.14229167


In [40]:
df_procedures = pd.read_csv(r'S:\NCATS\Clinical_Profiles\clean_data\Diabetes\jh_diabetes_procedure.txt',sep='|',encoding='Latin-1',
                           dtype=procedure_dtypes)

In [15]:
print(sys.getsizeof(df_procedures)*10**(-9))

1.067732568


### Calculating Lab Information

#### Lesson learned: grab patient IDs from demographics and then drop not needed columns, not all patients will have all encounter types, e.g. labs, medications, etc.

In [5]:
df_labs_full = df_labs.merge(df_demographics, on='PatientID', how='right')

In [8]:
df_labs_full.head()

,PatientID,EncounterID,Lab_Name,Base_Name,Ordering_datetime,Result_datetime,Result_numeric,Loinc_Code,LONG_COMMON_NAME,status,Category,GroupId,unit,range,DOB,Gender,Race,Ethnicity
0,65266,296268.0,Est GFR NonAfrAm(MDRD Eqn),GFRNA,2016-09-14,2016-09-14,13.0,48642-3,Glomerular filtration rate/1.73 sq M predicted...,ACTIVE,NaN,NaN,mL/min/1.73 sqm,NaN,1936-11-05,Male,White or Caucasian,Not Hispanic or Latino
1,65266,296268.0,Mean Corpus Hgb Conc,MCHC,2016-09-14,2016-09-14,32.4,786-4,Erythrocyte mean corpuscular hemoglobin concen...,ACTIVE,NaN,NaN,g/dL,"31.5,35.7",1936-11-05,Male,White or Caucasian,Not Hispanic or Latino
2,65266,296268.0,Prothrombin Time,LABPT,2016-09-14,2016-09-14,11.2,5964-2,Prothrombin time (PT) in Blood by Coagulation ...,ACTIVE,NaN,NaN,Seconds,NaN,1936-11-05,Male,White or Caucasian,Not Hispanic or Latino
3,65266,296268.0,Calcium,CALCIUM,2016-09-14,2016-09-14,8.3,17861-6,Calcium [Mass/volume] in Serum or Plasma,ACTIVE,Mass-Molar conversion,LG49864-8,mg/dL,"9,10.5",1936-11-05,Male,White or Caucasian,Not Hispanic or Latino
4,65266,296268.0,Hematocrit,HCT,2016-09-14,2016-09-14,34.6,4544-3,Hematocrit [Volume Fraction] of Blood by Autom...,ACTIVE,NaN,NaN,%,"34.0,46.6",1936-11-05,Male,White or Caucasian,Not Hispanic or Latino


In [9]:
(len(df_labs_full)-len(df_labs) )

19695

In [6]:
df_labs_full.drop(['Result_datetime','Base_Name','status','Category','GroupId'],axis=1,inplace=True)

In [17]:
print(sys.getsizeof(df_labs_full)*10**(-9))

0.54134634


In [11]:
code = df_labs_full.Loinc_Code.unique().dropna()

In [12]:
code[0]

'48642-3'

In [13]:
count = df_labs_full.Loinc_Code.value_counts()

In [17]:
count.index[0]

'41653-7'

In [24]:
df_labs_full['orderYear'] = pd.to_datetime(df_labs_full.Ordering_datetime).dt.year

In [25]:
frequencyPerYear = df_labs_full.groupby(['Loinc_Code','PatientID','orderYear']).PatientID.size().groupby(['Loinc_Code','orderYear']).aggregate(np.mean)

In [26]:
frequencyPerYear.head(20)

Loinc_Code  orderYear
12962-7     2013.0       1.000000
            2015.0       3.138728
            2016.0       5.480364
            2017.0       6.231171
            2018.0       4.392857
13457-7     2015.0       1.000000
            2016.0       1.511211
            2017.0       1.645030
            2018.0       1.124031
14749-6     2015.0       3.654088
            2016.0       3.913710
            2017.0       3.214286
1501-6      2016.0       1.000000
            2017.0       1.000000
            2018.0       1.000000
1530-5      2016.0       1.000000
            2017.0       1.000000
            2018.0       1.000000
1558-6      2015.0       1.800000
            2016.0       1.478873
Name: PatientID, dtype: float64

In [27]:
%time correlatedLabsCoefficients = df_labs_full.groupby('Loinc_Code').Result_numeric.apply(lambda x: pd.Series(x.values)).unstack().transpose().corr()

Wall time: 12.6 s


In [28]:
correlatedLabsCoefficients

Loinc_Code,12962-7,13457-7,14749-6,1501-6,1530-5,1558-6,1742-6,17856-6,17861-6,19123-9,...,1514-9,20437-0,14328-9,20636-7,1504-0,21395-9,2089-1,1507-3,12610-2,12646-6
Loinc_Code,,,,,,,,,,,,,,,,,,,,,
12962-7,1.000000,-0.016985,-0.000940,0.203315,0.287606,0.033756,-0.000728,0.003266,0.003870,-0.001075,...,0.101912,-0.130665,NaN,0.036454,0.142654,1.000000,NaN,0.244046,1.0,1.0
13457-7,-0.016985,1.000000,-0.028226,-0.387408,-0.685994,-0.016055,-0.010830,-0.013962,0.000838,-0.013399,...,-0.003808,-0.157846,NaN,-0.308754,0.087457,-0.999508,NaN,-0.167683,-1.0,-1.0
14749-6,-0.000940,-0.028226,1.000000,0.546596,0.048621,0.021765,-0.008217,0.029668,-0.015382,-0.004012,...,-0.419251,0.131162,NaN,0.265095,-0.100739,0.640829,NaN,0.078541,1.0,1.0
1501-6,0.203315,-0.387408,0.546596,1.000000,0.269193,0.289520,-0.494264,-0.326489,-0.268258,0.039099,...,-0.209691,-0.237103,NaN,-0.464204,-0.519649,0.971342,NaN,0.729974,1.0,1.0
1530-5,0.287606,-0.685994,0.048621,0.269193,1.000000,0.448599,0.289549,-0.362050,0.382696,0.211496,...,0.288036,-0.467081,NaN,0.225421,-0.096246,0.997469,NaN,-0.089038,1.0,1.0
1558-6,0.033756,-0.016055,0.021765,0.289520,0.448599,1.000000,0.003740,0.062195,-0.051122,0.031534,...,-0.363631,-0.298953,NaN,0.184457,0.521018,0.989746,NaN,-0.131440,1.0,1.0
1742-6,-0.000728,-0.010830,-0.008217,-0.494264,0.289549,0.003740,1.000000,0.089354,-0.006154,-0.005218,...,0.687559,-0.081835,NaN,0.226102,-0.114703,-0.356800,NaN,-0.220175,-1.0,-1.0
17856-6,0.003266,-0.013962,0.029668,-0.326489,-0.362050,0.062195,0.089354,1.000000,-0.005396,0.034263,...,0.691729,-0.242983,NaN,-0.367318,0.082787,-0.121972,NaN,-0.476661,-1.0,-1.0
17861-6,0.003870,0.000838,-0.015382,-0.268258,0.382696,-0.051122,-0.006154,-0.005396,1.000000,0.014924,...,0.148041,0.214847,NaN,0.016933,-0.077096,-0.821132,NaN,0.217740,-1.0,-1.0


In [29]:
abscorrelation = correlatedLabsCoefficients.abs()

In [30]:
fractionOfSubjects = df_labs_full.groupby(['Loinc_Code']).PatientID.nunique()/df_labs_full.PatientID.nunique()

In [31]:
fractionOfSubjects

Loinc_Code
12962-7    0.554395
13457-7    0.009537
14749-6    0.016526
1501-6     0.000084
1530-5     0.000111
1558-6     0.004734
1742-6     0.635424
17856-6    0.008938
17861-6    0.675605
19123-9    0.218264
1920-8     0.631860
1975-2     0.625024
1995-0     0.003606
2000-8     0.016526
20436-2    0.000766
2085-9     0.477320
2093-3     0.476805
2339-0     0.111979
2341-6     0.061760
2345-7     0.654721
25428-4    0.016429
2565-0     0.001991
2571-8     0.471264
2601-3     0.004483
2777-1     0.169145
2823-3     0.202587
2885-2     0.630886
2951-2     0.678515
3049-4     0.002088
3094-0     0.202893
             ...   
718-7      0.596539
731-0      0.005123
736-9      0.043188
737-7      0.062707
744-3      0.062721
777-3      0.596288
785-6      0.596483
786-4      0.596469
787-2      0.596483
788-0      0.596455
789-8      0.596497
15087-0    0.000139
17863-2    0.000167
20438-8    0.000599
20448-7    0.003119
2729-2     0.001462
21394-2    0.000084
50206-2    0.000780
17864-0  

In [32]:
units = df_labs_full.groupby(['Loinc_Code']).unit.unique()

In [18]:
minimum = df_labs_full.groupby(['Loinc_Code']).Result_numeric.min()
maximum = df_labs_full.groupby(['Loinc_Code']).Result_numeric.max()
mean = df_labs_full.groupby(['Loinc_Code']).Result_numeric.mean()
median = df_labs_full.groupby(['Loinc_Code']).Result_numeric.median()
stdDev = df_labs_full.groupby(['Loinc_Code']).Result_numeric.std()
nthDecile = df_labs_full.groupby('Loinc_Code').Result_numeric.quantile([0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9])

In [34]:
def percentile(n):
    def percentile_(x):
        return x.quantile(n*0.01)
    percentile_.__name__ = '%s' % n
    return percentile_

In [57]:
stats = (df_labs_full.groupby(['Loinc_Code'])
       .Result_numeric.agg(['min','max', 'mean','median','std',
                           percentile(10), percentile(20), percentile(30),
                           percentile(40), percentile(50), percentile(60),
                           percentile(70), percentile(80), percentile(90)]))

In [15]:
df_labs_full.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9496504 entries, 0 to 9496503
Data columns (total 13 columns):
PatientID            int64
EncounterID          float64
Lab_Name             category
Ordering_datetime    datetime64[ns]
Result_numeric       float64
Loinc_Code           category
LONG_COMMON_NAME     category
unit                 category
range                category
DOB                  datetime64[ns]
Gender               category
Race                 category
Ethnicity            category
dtypes: category(8), datetime64[ns](2), float64(2), int64(1)
memory usage: 516.2 MB


In [8]:
df_labs_full['range_high'] = (pd.to_numeric(df_labs_full.range.dropna()
               .astype('str').str.split(',',expand=True)[1]).astype('float'))

df_labs_full['range_low'] = (pd.to_numeric(df_labs_full.range.dropna()
               .astype('str').str.split(',',expand=True)[0]).astype('float'))

In [37]:
def fracsAboveBelowNormal(x):
    aboveNorm = np.divide(np.sum(x.Result_numeric > x.range_high), x.Result_numeric.size)
    belowNorm = np.divide(np.sum(x.Result_numeric < x.range_low), x.Result_numeric.size)
    return pd.Series({'aboveNorm':aboveNorm, 'belowNorm':belowNorm})

In [39]:
%%time
aboveBelowNorm = (df_labs_full.groupby(['Loinc_Code'])
 .apply(fracsAboveBelowNormal))


Wall time: 6.8 s


In [31]:
aboveBelowNorm.aboveNorm

Loinc_Code
12962-7    0.000000
13457-7    0.241811
14749-6    0.873013
1501-6     0.428571
1530-5     0.444444
1558-6     0.773314
1742-6     0.363134
17856-6    0.000000
17861-6    0.010087
19123-9    0.166550
1920-8     0.152295
1975-2     0.081893
1995-0     0.000000
2000-8     0.012492
20436-2    0.000000
2085-9     0.062657
2093-3     0.224795
2339-0     0.830832
2341-6     0.000000
2345-7     0.783221
25428-4    0.000000
2565-0     0.000000
2571-8     0.389464
2601-3     0.155583
2777-1     0.146064
2823-3     0.049996
2885-2     0.009263
2951-2     0.035949
3049-4     0.564417
3094-0     0.241634
             ...   
718-7      0.012931
731-0      0.131579
736-9      0.000000
737-7      0.148266
744-3      0.141454
777-3      0.062345
785-6      0.042430
786-4      0.016729
787-2      0.106432
788-0      0.389591
789-8      0.037296
15087-0    0.000000
17863-2    0.230769
20438-8    0.000000
20448-7    0.311178
2729-2     0.000000
21394-2    0.000000
50206-2    0.000000
17864-0  

**NOTE: Less than a minute to calculate all necessary lab information (~9 million rows)**

#### Printing out first 10 results from each calculated field as an example
*If you copy this file, feel free to remove .head(10) to see all results, by default pandas groupby sorts alphanumerically*

In [34]:
code.head(10)

AttributeError: 'Categorical' object has no attribute 'head'

In [ ]:
count.head(10)

In [ ]:
frequencyPerYear.head(10)

In [ ]:
correlatedLabsCoefficients.head(10)

In [ ]:
abscorrelation.head(10)

In [ ]:
fractionOfSubjects.head(10)

In [ ]:
units.head(10)

In [ ]:
minimum.head(10)

In [ ]:
maximum.head(10)

In [ ]:
mean.head(10)

In [ ]:
median.head(10)

In [ ]:
stdDev.head(10)

In [ ]:
nthDecile.head(20)

### Define Correlation Functions Needed for Categorical Data

In [ ]:
def cramers_v(df, x, y):
    confusion_matrix = (df.groupby([x,y])[y].size().unstack().fillna(0).astype(int))
    chi2 = ss.chi2_contingency(confusion_matrix)[0]
    n = confusion_matrix.sum().sum()
    phi2 = chi2/n
    r,k = confusion_matrix.shape
    phi2corr = max(0, phi2-((k-1)*(r-1))/(n-1))
    rcorr = r-((r-1)**2)/(n-1)
    kcorr = k-((k-1)**2)/(n-1)
    return np.sqrt(phi2corr/min((kcorr-1), (rcorr-1)))

def uncertainty_coefficient(df, x, y):
    df2 = df[[x,y]]
    total = len(df2.dropna())
    p_y = (df.groupby([y], sort=False)[y].size()/total).reindex(index=p_xy.index, level=1)

    s_xy = sum(p_xy * (p_y/p_xy).apply(math.log))

    p_x = df.groupby([x], sort=False)[x].size()/total
    s_x = ss.entropy(p_x)
    if s_x == 0:
        return  1
    else:
        return ((s_x - s_xy) / s_x)

def correlation_ratio(df, x, y):
    df2 =  df.groupby([x],sort=False)[y].agg([np.size,np.mean])
    ybar = df[y].mean()
    numerator = np.nansum(np.multiply(df2['size'],np.square(df2['mean']-ybar)))
    ssd = np.square(df[y]-ybar)
    #ssd = df.groupby([x,y],sort=False)[y].apply(lambda y: np.nansum(np.square(y-ybar)))
    denominator = np.nansum(ssd)
    if numerator == 0:
        return 0.0
    else:
        return np.sqrt(numerator/denominator)   

### Join All DataFrames to "Correlate Everything to Everything"

In [29]:
df = (df_labs.merge(df_diagnoses_hpo, on='PatientID')
             .merge(df_encounter, on=['PatientID','EncounterID'], how='outer')
             .merge(df_meds, on=['PatientID','EncounterID'], how='outer'))

### Define Categorical Fields

In [30]:
categoricals = ['Lab_Name','Base_Name','Loinc_Code','LONG_COMMON_NAME','Category','GroupId','icd_10','icd_name',
                'hpo','hpo_term','Encounter_type','Medication_Name','Dose','Route','Frequency','RXNorm',
               'Therapeutic_Class','Pharmaceutical_Class','Pharmaceutical_Subclass']

## Work in Progress...
#### Need to Define Correlations More Precisely

## Will Add in Other Fields & Their Calculated Results Shortly.....

### Medications

In [35]:
df_meds_full = df_meds.merge(df_demographics, on='PatientID', how='outer')

In [36]:
(len(df_meds_full) - len(df_meds))

17715

**Why is Medication Name nunique() > RXNorm nunique() ?**

In [37]:
medication = df_meds_full.RXNorm.unique()

uniqDropNA = lambda x: np.unique(x.dropna())

dosageInfo = df_meds_full.groupby('RXNorm').agg({'Route':uniqDropNA, 'Dose':uniqDropNA,'Quantity':uniqDropNA})#[['Route','Dose','Quantity']].apply(np.unique)
#dose = df_meds_full.groupby('RXNorm')['Dose'].unique()
#quantity = df_meds_full.groupby('RXNorm')['Quantity'].unique()

# How to calculate rateRatio?!

#treatmentDuration says need clarification in model!

df_meds_full['startYear'] = pd.to_datetime(df_meds_full.Start_date).dt.year

frequencyPerYear = df_meds_full.groupby(['RXNorm','startYear','PatientID']).PatientID.count().groupby(['RXNorm','startYear']).mean()

fractionOfSubjects = df_meds_full.groupby(['RXNorm']).PatientID.nunique()/df_meds_full.PatientID.nunique()

#correlatedLabsCoefficients = df_labs.groupby('LONG_COMMON_NAME').Result_numeric.apply(lambda x: pd.Series(x.values)).unstack().transpose().corr()

#abscorrelation = correlatedLabsCoefficients.abs()

In [38]:
dosageInfo

,Route,Dose,Quantity
RXNorm,,,
38.0,[Oral],"[1 tablet, 1.25 mg, 2.4 mg, 2.5, 2.5 mg, 5, 5 mg]",[]
44.0,[],[800 mg/m2/dose],[]
90.0,[Oral],"[1, 1 capsule, 1 each, 1 g, 1 tablet, 1000 mg,...",[]
94.0,[],"[1 capsule, 1 tablet, 100 mg]",[]
99.0,"[G-Tube, Oral]","[.5, 1, 1 g, 100, 100 mg/kg/dose, 1000 mg, 2, ...",[]
103.0,[Oral],"[1 tablet, 100, 100 mg, 125, 125 mg, 18.75 mg/...",[]
110.0,[],"[.5 Applicatorful, 1 Applicatorful, 1 Tube]",[]
149.0,[Oral],"[200, 200 mg, 400 mg]",[]
161.0,"[G-Tube, Intravenous, J-Tube, Nasoduodenal Tub...","[.5, .5 tablet, .5-1 tablet, 1, 1 capsule, 1 e...",[]


### Diagnosis

In [39]:
df_diagnoses_hpo_full = df_diagnoses_hpo.merge(df_demographics, on='PatientID', how='outer')

In [40]:
(len(df_diagnoses_hpo_full) - len(df_diagnoses_hpo))

16797

In [41]:
code = df_diagnoses_hpo_full.icd_10.unique()

df_diagnoses_hpo_full['entryYear'] = pd.to_datetime(df_diagnoses_hpo_full.Entry_Date).dt.year

frequencyPerYear = df_diagnoses_hpo_full.groupby(['icd_10','entryYear','PatientID']).PatientID.count().groupby(['icd_10','entryYear']).mean()

fractionOfSubjects = df_diagnoses_hpo_full.groupby(['icd_10']).PatientID.nunique()/df_diagnoses_hpo_full.PatientID.nunique()

In [42]:
frequencyPerYear

icd_10   entryYear
A46      2016.0       1.000000
         2017.0       1.000000
         2018.0       1.000000
B00.4    1900.0       1.000000
         2005.0       1.000000
         2013.0       1.000000
         2017.0       1.000000
         2018.0       1.000000
         2019.0       1.000000
B35.1    1899.0       1.000000
         1900.0       1.000000
         2004.0       1.000000
         2005.0       1.000000
         2006.0       1.000000
         2007.0       1.000000
         2008.0       1.000000
         2009.0       1.000000
         2010.0       1.000000
         2011.0       1.000000
         2012.0       1.000000
         2013.0       1.019231
         2014.0       1.014493
         2015.0       1.023077
         2016.0       1.011429
         2017.0       1.009615
         2018.0       1.026549
         2019.0       1.050000
B96.20   2013.0       1.000000
         2014.0       1.000000
         2015.0       1.000000
                        ...   
L85.2    2016.0     

### Procedures

In [41]:
df_procedures_full = df_procedures.merge(df_demographics, on='PatientID', how='right')

In [42]:
df_procedures_full.drop(['DOB','Gender','Race','Ethnicity'], axis=1, inplace=True)

**I need the encounter table to get a date**

In [43]:
encounter_dtypes = {'PatientID': np.int64, 'EncounterID': np.int64, 'Encounter_type': 'category'}
encounter_date = ['Encounter_date']
df_encounter = pd.read_csv(r'S:\NCATS\Clinical_Profiles\clean_data\Diabetes\jh_diabetes_encounter.txt',sep='|',
                             dtype=encounter_dtypes, parse_dates=encounter_date)

In [48]:
print(sys.getsizeof(df_encounter)*10**(-9))

0.03001293


In [45]:
df_procedures_full = df_procedures_full.merge(df_encounter, on=['EncounterID','PatientID'], how='left')

In [46]:
print(sys.getsizeof(df_procedures_full)*10**(-9))

1.7153255120000002


In [47]:
(len(df_procedures_full) - len(df_procedures))

641

In [48]:
df_procedures_full.columns

Index(['PatientID', 'EncounterID', 'Procedure_ID', 'Procedure_Code',
       'Procedure_Name', 'Encounter_date', 'Encounter_type'],
      dtype='object')

In [54]:
# Oops don't need extra patient column
len(df_procedures_full.PatientID_x.dropna()) - len(df_procedures_full.PatientID_y.dropna())

641

In [55]:
df_procedures_full.drop('PatientID_y',axis=1,inplace=True)

In [56]:
df_procedures_full.rename(columns={'PatientID_x': 'PatientID'}, inplace=True)

procedure_dtypes = {'PatidentID':np.int64,'EncounterID':np.int64, 'Procedure_ID':np.int64,'Procedure_Code':'category',
                   'Procedure_Name':'category'}

In [57]:
code = df_procedures_full.Procedure_Code.unique()

df_procedures_full['encounterYear'] = pd.to_datetime(df_procedures_full.Encounter_date).dt.year

frequencyPerYear = (df_procedures_full.groupby(['Procedure_Code','encounterYear','PatientID']).PatientID.count()
                                        .groupby(['Procedure_Code','encounterYear']).mean())

fractionOfSubjects = df_procedures_full.groupby(['Procedure_Code']).PatientID.nunique()/df_procedures_full.PatientID.nunique()

In [58]:
fractionOfSubjects

Procedure_Code
01996    0.004748
03000    0.077465
0346T    0.000306
0500F    0.000849
0501F    0.000933
0502F    0.001671
0503F    0.001921
10022    0.004274
10030    0.000585
10060    0.002952
10061    0.002590
10140    0.001086
10160    0.001086
11005    0.000237
11042    0.004135
11043    0.002200
11044    0.002186
11046    0.000821
11047    0.000306
11055    0.001657
11056    0.007240
11057    0.007462
11100    0.014995
11101    0.005653
11200    0.000905
11402    0.001100
11403    0.000640
11404    0.000195
11406    0.000418
11443    0.000111
           ...   
27645    0.000014
33910    0.000014
33915    0.000014
34421    0.000014
35694    0.000014
35741    0.000014
37650    0.000014
37700    0.000014
40840    0.000014
43116    0.000014
44208    0.000014
45000    0.000014
49419    0.000014
57320    0.000014
58240    0.000014
61450    0.000014
61650    0.000014
61651    0.000014
62146    0.000014
62200    0.000014
27396    0.000014
31294    0.000014
39599    0.000014
43100    0.00